# Animal Breed Classification

In [14]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D,Input, Lambda, Dense, Flatten, Dropout, MaxPool2D
from keras.models import Model

# I have used transfer learning models from keras applications and, i was able to get the higher accuracy with InceptionResNetV2

#from keras.applications.vgg16 import VGG16
#from keras.applications import ResNet152V2
#from keras.applications import InceptionResNetV2
#from keras.applications.inception_v3 import InceptionV3
from keras.applications import NASNetLarge

from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Sequential

import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import numpy as np
import shutil
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
#Highest Score : 91.71429

In [13]:
#To limit GPU usage

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [15]:
IMAGE_SIZE = [331, 331]

train_path = 'TRAIN'
valid_path = 'TEST'

In [ ]:
rootdir= 'TRAIN' #path of the original folder

classes = ['abyssinian','american_bulldog','american_pit_bull_terrier','basset_hound','beagle','bengal','birman','bombay','boxer','british_shorthair','chihuahua','egyptian_mau','english_cocker_spaniel','english_setter','german_shorthaired','great_pyrenees','havanese','japanese_chin','keeshond','leonberger','maine_coon','miniature_pinscher','newfoundland','persian','pomeranian','pug','ragdoll','russian_blue','saint_bernard','samoyed','scottish_terrier','shiba_inu','siamese', 'sphynx','staffordshire_bull_terrier','wheaten_terrier', 'yorkshire_terrier']

for i in classes:
    os.makedirs(rootdir +'/train/' + i)

    os.makedirs(rootdir +'/test/' + i)

    source = rootdir + '/' + i

    allFileNames = os.listdir(source)

    np.random.shuffle(allFileNames)

    test_ratio = 0.08

    train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)* (1 - test_ratio))])

    train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]

    for name in train_FileNames:
        shutil.copy(name, rootdir +'/train/' + i)

    for name in test_FileNames:
        shutil.copy(name, rootdir +'/test/' + i) 

In [16]:
# Fetching the required model with required input_shape and include_top as false to neglect output layers

nasnet = NASNetLarge(input_shape=(331,331,3), weights='imagenet', include_top=False)

In [17]:
#Since, we dont need to train the pre-trained layers of the model in transfer learning, we set the trainable property to be false

for layer in nasnet.layers:
    layer.trainable = False

In [18]:
#Folders represents classes, I have my training image folders under the 'TRAIN' folder

folders = glob('TRAIN/*')
folders

['TRAIN\\abyssinian',
 'TRAIN\\american_bulldog',
 'TRAIN\\american_pit_bull_terrier',
 'TRAIN\\basset_hound',
 'TRAIN\\beagle',
 'TRAIN\\bengal',
 'TRAIN\\birman',
 'TRAIN\\bombay',
 'TRAIN\\boxer',
 'TRAIN\\british_shorthair',
 'TRAIN\\chihuahua',
 'TRAIN\\egyptian_mau',
 'TRAIN\\english_cocker_spaniel',
 'TRAIN\\english_setter',
 'TRAIN\\german_shorthaired',
 'TRAIN\\great_pyrenees',
 'TRAIN\\havanese',
 'TRAIN\\japanese_chin',
 'TRAIN\\keeshond',
 'TRAIN\\leonberger',
 'TRAIN\\maine_coon',
 'TRAIN\\miniature_pinscher',
 'TRAIN\\newfoundland',
 'TRAIN\\persian',
 'TRAIN\\pomeranian',
 'TRAIN\\pug',
 'TRAIN\\ragdoll',
 'TRAIN\\russian_blue',
 'TRAIN\\saint_bernard',
 'TRAIN\\samoyed',
 'TRAIN\\scottish_terrier',
 'TRAIN\\shiba_inu',
 'TRAIN\\siamese',
 'TRAIN\\sphynx',
 'TRAIN\\staffordshire_bull_terrier',
 'TRAIN\\wheaten_terrier',
 'TRAIN\\yorkshire_terrier']

In [19]:
# Flattening the final layer output, to get a vector

x = Flatten()(nasnet.output)

In [20]:
#Creating a dense layer with number of nodes equal to number of folders(which specify classes) for our output layer, Since it is multiclass we use softmax activation function

prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=nasnet.input, outputs=prediction)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_261 (Activation)     (None, 165, 165, 96) 0           stem_bn1[0][0]                   
____________________________________________________________________________________________

In [ ]:
"""
possible data augmentation parameters(sample)

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.1, 0.9]
)

"""

In [21]:
#Compiling the model with specified optimizer and loss functions

model.compile(Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
#Reading the data from our train and test folders 

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    'TRAIN',
    target_size=(331,331),
    batch_size=32,
    class_mode='categorical'
)

check_set = test_datagen.flow_from_directory('TEST',
                                            target_size = (331, 331))
"""
Use it if you want a validation set

validation_generator = train_datagen.flow_from_directory(
    'testy',
    target_size=(331,331),
    batch_size=32,
    class_mode='categorical'
)"""

Found 5963 images belonging to 37 classes.
Found 1478 images belonging to 37 classes.


In [1]:
#Fitting the model over our data with callbacks

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=2)
filepath = 'NasNetLarge(1).h5'
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1, save_best_only=True,mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5, patience=2, verbose=1, mode='max',min_lr=0.00001)
op = model.fit_generator(
  train_generator,
  validation_data = validation_generator,
  epochs=5,
  callbacks = [checkpoint,reduce_lr,es]
)

Training


# Custom Model

In [ ]:
kernel_size = (3,3)
pool_size = (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3
IMAGE_SIZE = 224

batch_size = 32

In [ ]:
model = Sequential()
model.add(Conv2D(first_filters,kernel_size,activation='relu',input_shape = (IMAGE_SIZE,IMAGE_SIZE,3))) #This 3, would be a 1 if the images are gray
model.add(Conv2D(first_filters,kernel_size,activation='relu'))
model.add(Conv2D(first_filters,kernel_size,activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters,kernel_size,activation='relu'))
model.add(Conv2D(second_filters,kernel_size,activation='relu'))
model.add(Conv2D(second_filters,kernel_size,activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters,kernel_size,activation='relu'))
model.add(Conv2D(third_filters,kernel_size,activation='relu'))
model.add(Conv2D(third_filters,kernel_size,activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(dropout_dense))
model.add(Dense(37, activation='softmax'))

model.compile(Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
filepath = 'Inceptionv3(2).h5'
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1, save_best_only=True,mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5, patience=3, verbose=1, mode='max',min_lr=0.00001)
callsback = [checkpoint, reduce_lr]

fitting = model.fit(train_generator, steps_per_epoch= len(train_generator), validation_data=validation_generator, validation_steps=len(validation_generator), epochs=30, verbose=1)

# Visualization and predictions

In [ ]:
# plot the loss
plt.plot(op.history['loss'], label='train loss')
plt.plot(op.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(op.history['accuracy'], label='train acc')
plt.plot(op.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#Snippet to save trained model

from tensorflow.keras.models import load_model

model.save('model_IncResnetV2.h5')

In [ ]:
#Function for prediction

def predictImage(filename):
    img1 = image.load_img('TEST/'+filename,target_size=(331,331))
    Y = image.img_to_array(img1)
    Y=Y/255
    X = np.expand_dims(Y,axis=0)
    val = np.argmax(model.predict(X), axis=1)
    return val

In [ ]:
classes = ['abyssinian','american_bulldog','american_pit_bull_terrier','basset_hound','beagle','bengal','birman','bombay','boxer','british_shorthair','chihuahua','egyptian_mau','english_cocker_spaniel','english_setter','german_shorthaired','great_pyrenees','havanese','japanese_chin','keeshond','leonberger','maine_coon','miniature_pinscher','newfoundland','persian','pomeranian','pug','ragdoll','russian_blue','saint_bernard','samoyed','scottish_terrier','shiba_inu','siamese', 'sphynx','staffordshire_bull_terrier','wheaten_terrier', 'yorkshire_terrier']
ans=[]
for i in range(1500):
    res = predictImage(str(i+1)+'.jpg')
    ans.append(classes[res[0]])

In [ ]:
#Custom tester utility with visualization

img1 = image.load_img('TEST/1498.jpg',target_size=(331,331))
plt.imshow(img1)
Y = image.img_to_array(img1)
Y=Y/255
X = np.expand_dims(Y,axis=0)
val = np.argmax(model.predict(X), axis=1)
print(model.predict(X))
classes[val[0]]

In [ ]:
naming = []
for i in range(len(ans)):
    naming.append(str(i+1)+'.jpg')

In [ ]:
combine = list(zip(naming,ans))
result = pd.DataFrame(combine,columns=['Filename','Class'])

In [ ]:
result.to_csv('NasNetLarge(6).csv',index=False)

In [ ]:
paths = []
for i in range(len(ans)):
    if ans[i] == 'staffordshire_bull_terrier':
        paths.append('TEST/'+str(naming[i]))
        img = image.load_img('TEST/'+str(naming[i]),target_size=(224,224))
        plt.imshow(img)
        continue